In [39]:
import cv2
import numpy as np
import time
import sys
import os



In [40]:
brushThickness = 10
eraserThickness = 40

In [41]:
sys.path.append('D:/ComputerVision_MultipleProj/HandTrackingProject')
import HandTrackingModule as htm


In [44]:
folderPath = 'Header'
myList = os.listdir(folderPath)
print(myList)
overlayList = []

['1.jpg', '2.jpg', '3.jpg', '4.jpg']


In [45]:
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    overlayList.append(image)
    
    
    

In [46]:
print(len(overlayList))

4


In [50]:
header = overlayList[0]
drawColor = (0,0,255)

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
detector = htm.handDetector(detectionCon=0.80)
xp,yp = 0,0
imgCanvas = np.zeros((720,1280,3),np.uint8)
while cap.isOpened():
    # 1. import image
    success,img = cap.read()
    img = cv2.flip(img,1)
    
    if success == True:
        # 2. Find Hand Landmarks
        img = detector.findHands(img)
        lmList = detector.findPosition(img, draw=False)
        
        if len(lmList)!= 0:
            
#             print(lmList)
            
            # tip of index and middle finger
            x1,y1 = lmList[8][1:]
            x2,y2 = lmList[12][1:]
        
            # 3. Check which fingers are up
            fingers = detector.fingersUp()
#             print(fingers)
            
            # 4. If selection mode - Two fingers are up
            if fingers[1] and fingers[2]:
                xp,yp = 0,0
                print("selection mode")
                # Check the click in header. If 1st selection then 1st image and so on
                if y1<125:
                    if 50<x1<250:
                        header = overlayList[0]
                        drawColor = (0,0,255)
                    elif 250<x1<450:
                        header = overlayList[1]
                        drawColor = (255,0,0)
                    elif 500< x1 < 700:
                        header = overlayList[2]
                        drawColor = (0,255,0)
                    elif 750 < x1 < 950:
                        header = overlayList[3]
                        drawColor = (0,0,0)
                cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawColor,cv2.FILLED)
                        
#                     else: 
#                         header = overlayList[0]

            # 5. If drawing mode - Index finger is up
            if fingers[1] and fingers[2] == False:
                cv2.circle(img,(x1,y1),10,drawColor,cv2.FILLED)
                print("Drawing mode")
                if xp==0 and yp==0:
                    xp,yp = x1,y1
                
                if drawColor == (0,0,0):
                    cv2.line(img,(xp,yp),(x1,y1), drawColor,eraserThickness)
                    cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,eraserThickness)
                    
                else:
                
                    cv2.line(img,(xp,yp),(x1,y1), drawColor,brushThickness)
                    cv2.line(imgCanvas,(xp,yp),(x1,y1),drawColor,brushThickness)

                xp,yp = x1,y1
                
                
                
        imgGray = cv2.cvtColor(imgCanvas,cv2.COLOR_BGR2GRAY)
        _, imgInv = cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
        imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
        img = cv2.bitwise_and(img,imgInv)
        img = cv2.bitwise_or(img,imgCanvas)
        
        
        
        
        # setting the header image
        img[0:125,0:1280] = header
#         img = cv2.addWeighted(img,0.5,imgCanvas,0.5,0)
        cv2.imshow("Image",img)
        cv2.imshow("Canvas",imgCanvas)
        if cv2.waitKey(1) == ord('q'):
            break            
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

        
    

In [ ]:
! git add virtualPainter.ipynb readme.md header
! git commit -m "13:33/05-01-2022"
! git push origin main